## Mutants:
#### since the discovery of their existence they have been regarded with fear, suspicious, often hatred
Implement a rMPIPD where strategies are allowed to mutate. The goal is to simulate the effect of genetic mutations and the effect of natural selection. A parameter (gene) should encode the attidue of an individual to cooperate, such gene can mutate randomly and the corresponding phenotype should compete in the MPIPD such that the best-fitted is determined.  


## Solution
The most compact way to implement this using the previous results and defined elements is to implement a more complex structure which can behave differently according to a memory element. This can be done using a class to manage Mutants: a mutant should have a genome which contains the gene that allows him to mutate and determine the nature of the mutation. To simulate this, the Mutant is a guy that is allowed to use some of the previous defined strategies, each one with a defined probability that changes during the match or tournament; to do so it is always allowed to have access to nice_guy and bad_guy, and from those can create mainly_nice, mainly_bad and random. At every moment the class can be called to get a move and the evolve function can be invoked to make it change.

In [151]:
class mutant :

    # CONSTRUCTOR
    # every mutant is created with a name and a list of tuples, containing accessible strategies and their initial weight
    def __init__( self, name, starting_strategy ) : 
        self.name = name
        # the series of trategies that can be chosen, starting from a list of strategies and their probabilities
        self.strategy_dic = { i[0] : strategies[i[0]] for i in starting_strategy } 
        # NiceGuy and BadGuy incarnate the basic behaviour, thus are always included as last elements, if not already present
        self.strategy_dic[ 'NiceGuy' ] = nice_guy
        self.strategy_dic[ 'BadGuy' ] = bad_guy
        # this process creates an internal dictionary similar to strategies but with limited entries, useful for later cycles
        # the gene in rapresented by the strategy list, a list of values that determine which strategy use to create the next move
        # this is rapresented by a normalized range of values chosen by a uniform distribution
        starting_weight = [ i[1] for i in starting_strategy ]
        self.strategy = np.array( starting_weight/np.sum(starting_weight) )
        # reshaping the weight array to consider added elements
        self.strategy.resize( len( self.strategy_dic ) )
        self.dic_keys = list( self.strategy_dic.keys() )

    # alternative constructor: it takes genome, a string of gene letters, the ripetition of each means a different probability
    @classmethod
    def from_genome( cls, genome ) :
        g_list = [ ( g, genome.count( g ) ) for g in list( cls.genes.keys() ) if ( genome.count( g ) ) ]
        total_g = sum( [ i[1] for i in g_list ] )
        strategy = [ ( cls.genes[ g[0] ] , float( g[1] ) / float( total_g )  ) for g in g_list ]
        return cls( genome, strategy )
    
    # REPR OUTPUT
    def __repr__( self ) :
        return f"<Mutant name:{self.name} strategy_dic:{self.strategy_dic} strategy:{self.strategy}>"

    # PRINTING OUTPUT
    def __str__( self ) :
        return f"Mutant class:\n   Name: {self.name}\n   Strategies in the genome: {[ i for i in self.strategy_dic]}\n   Gene values: {self.strategy}"

    # DEFINING HOW COPY BEHAVE, PREVENTING A SHALLOW COPY
    def __copy__( self ) :
        # returning a new object made from the same basic informations, name and list of couples of name of strategy used and its weight 
        return mutant( self.name, zip( self.dic_keys, self.strategy ) )
    
    
    # defining the generic function that returns a move for a mutant
    def move( self, round_number = 0, match_history = [[]], player_index = 0 ) :
        # generating a random value to chose the strategy
        u = npr.random( )
        # cumulative sum of weights
        cumulative = [ np.sum( self.strategy[ 0 : i + 1 ] ) for i in range( self.strategy.shape[ 0 ] ) ]
        # cycling over strategies weights
        for i in range( len( cumulative ) ) :
            if ( u < cumulative[ i ] ) : 
                return self.strategy_dic[ self.dic_keys[ i ] ]( round_number, match_history, player_index )    

    # function to change genome (strategy) corresponding to previous round results
    def mutate( self ) :
        # recovering existing strategy
        strat = self.strategy 
        
        # updating strategy: for each weight a new one is produced based on a gaussian
        # this makes small differences more likely; the gaussian is centered on the previous value and has little std
        # considering these are normalized probability their value is always less than 1
        std = 0.1
        # creating an array of normal distributed value centered in the previous values, given std and same shape as strategy 
        p = npr.normal( loc = strat, scale = std, size = strat.shape )
        # updating strategy only for those distributions that has positive outcome
        # this way genes that previously where 0. are less likely to sudden emerge
        strat[ p > 0. ] = p[ p > 0. ]
        # renormalize strategy so that move can be used with a uniform distribution
        self.strategy = strat / np.sum( strat )

    genes = {
            'N' : 'NiceGuy',
            'B' : 'BadGuy',
            'C' : 'TitForTat',
            'R' : 'ResentfulGuy',
            'T' : 'Thanos',
            'F' : 'TrustingGuy',
            'M' : 'MidResentful',
            'S' : 'ScammingGuy',
            'K' : 'ReverseTft',
            }
# the mutation can be encoded in a second moment by the usage of 
# @classmethod
# def mutate( cls, * x ) :
#     cls.strategy = x

In [175]:
# st = [ ( 'Thanos', 0.3 ), ( 'TitForTat', 0.7 ), ( 'NiceGuy', 1.) ]
# dic = { i : strategies[st[i][0]] for i in range(len(st)) }
# if ( not ( bad_guy in dic.values() ) ) : dic[ len(dic) ] = bad_guy
# print([ (i, dic[i])  for i in dic ])
# print( sum( np.array( [ 0.1, 0.5, 0.3 ] ) ) )

# import copy
# user = mutant( 'user', [ ( 'NiceGuy', 0.2 ), ( 'TitForTat', 0.5 ), ( 'Thanos', 0.1 ) ] )

# copy method
# unuser = copy.deepcopy( user )
# unuser.mutate()
# unuser.name = 'unuser'

# print( user )
# print( unuser )
# print( [ user.move() for i in range(10) ] )
# print( tit_for_tat( round_number = 0, match_history = [[]], player_index = 0 ) )
# print( list(strategies.keys())[0] )

#test with dictionary
# for i in range(10) : user.mutate()
# strategies[ 'user' ] = user.move

# mutate and move test
# for i in range(10) :
#     user.mutate()
#     print(user, user.move( ))
# for i in range(10) :
#     user.mutate()
#     print(user, user.move( ))

# print(user)
# user.mutate()
# print(user)
# user.mutate()
# print(user)
# user.mutate()
# print(user)
# user.mutate()
# print(user)
# user.mutate()
# print(user)
# user.mutate()
# print(user)

# print(user)
# print(unuser)

# for i in range(100):
#     user.mutate()
#     unuser.mutate()

# print(user)
# print(unuser)

# gen = mutant.from_genome( "NNN" )
# gen.mutate()
# print( gen )


# print ( match( 'NiceGuy', 'ScammingGuy', N_rounds = 10, M = Payoff ) )
# print ( match( 'BadGuy', 'ScammingGuy', N_rounds = 10, M = Payoff ) )
# print ( match( 'ResentfulGuy', 'ScammingGuy', N_rounds = 10, M = Payoff ) )
# print ( match( 'MainlyNice', 'ScammingGuy', N_rounds = 10, M = Payoff ) )
# print ( match( 'TitForTat', 'ScammingGuy', N_rounds = 10, M = Payoff ) )


# strategies[ gen.name ] = gen.move
# print(  match( gen.name, 'ScammingGuy', N_rounds = 10, M = Payoff ) )
# for i in range( 10 ) : gen.mutate()
# print(  match( gen.name, 'ScammingGuy', N_rounds = 10, M = Payoff ) )
# print( match( 'user', 'BadGuy', 20, M = Payoff ) )
# del strategies[ 'user' ]
# print( [ i for i in strategies ] )